# 앙상블  

>단일 모델의 f1_macro 결과  

- **LOF : 0.53**
- **ISF : 0.68**
- GMM : 0.39
- K-Means : 0.49
- One Class SVM : 0.34
- **Elliptic Envelope : 0.50** (데이터 스케일링하고 모델 학습시키면 성능 개선됨)  
- **AutoEncoder : 0.87**  

<br>

---

=> 단일 모델 중 성능이 좋았던 모델에 대해 앙상블  
(1) AE + EE + ISF  
(2) AE + EE + LOF  
(3) AE + EE + LOF + ISF  
(4) AE + LOF + ISF  

참고 : [깃허브](https://github.com/jailies/Dacon_Credit_Card_Fraud_Detection/blob/main/Ensemble/Ensemble.ipynb )

# import

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import pandas as pd
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# 데이터

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/daicon/train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('/content/drive/MyDrive/daicon/val.csv')
val_df = val_df.drop(columns=['ID'])

In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/daicon/test.csv')

In [6]:
val_x = val_df.drop(columns=['Class'])
val_y = val_df['Class'] # label

In [39]:
def get_pred_label(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [8]:
# StandardScaler 선언 및 Fitting
sdscaler = StandardScaler()
sdscaler.fit(train_df)

# 데이터 변환
sdscaled_data = sdscaler.transform(train_df)

# 데이터 프레임으로 저장
sdscaled_data = pd.DataFrame(sdscaled_data)

In [9]:
sdscaled_data

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.696317,-0.812481,1.178089,0.271798,-0.368309,1.349308,0.652511,0.210988,-1.380804,0.188955,...,0.345210,1.066111,1.431834,-1.136246,-0.633230,-0.288586,-0.137969,-0.166355,1.188563,-1.990839
1,-0.495358,-0.112967,1.191305,-0.608173,-0.007886,0.933476,0.192541,0.320444,-1.264291,-0.056156,...,-0.148280,0.007285,-0.297148,-1.938909,1.241563,-0.460803,0.157587,0.173998,0.143876,-1.990839
2,-0.218427,0.580982,0.755819,-0.116154,0.307498,-0.026206,0.390690,0.221649,-0.517216,-0.351540,...,-0.286720,-0.773425,-0.039431,-0.611606,-0.450853,0.220818,0.635984,0.229098,-0.346737,-1.990818
3,-0.330318,0.858041,0.711241,-0.345477,0.693558,0.317890,0.925884,-3.209968,0.563699,1.161326,...,2.693508,-1.402900,0.092477,-1.070930,-0.801717,-0.106743,-3.020466,-3.046182,-0.194717,-1.990712
4,-0.458462,0.172537,-0.082109,-0.189262,1.951788,2.793282,0.302611,0.719980,-0.355977,-0.387965,...,-0.099976,-0.370382,-0.319020,1.671161,0.714378,-0.798093,0.029990,0.401295,0.019821,-1.990712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113837,-6.415477,6.169704,-5.669274,-1.774215,-3.354334,-1.051879,-3.021567,4.640268,4.468824,8.074160,...,-1.306812,-2.162159,1.402375,-2.067159,3.432320,0.667510,5.233857,3.463283,-0.321352,1.645815
113838,0.965967,-0.087719,-0.674497,1.069597,-0.026160,-0.465046,0.153202,-0.208020,0.610333,0.107995,...,0.201179,0.876793,-0.064140,-0.086360,0.605158,-0.958774,0.046304,-0.113892,-0.116108,1.645857
113839,-0.124097,0.430608,0.260596,-0.325094,0.178465,-1.013702,0.767042,-0.171877,0.098905,-0.270613,...,-0.315282,-0.710635,0.441651,0.614549,-1.078550,0.235849,0.329761,0.229617,-0.339285,1.645899
113840,0.061566,0.563104,-0.371251,-0.524503,0.826191,-0.181210,0.670168,0.099151,-0.184366,-0.618512,...,-0.433467,-1.117010,0.081219,0.171137,-0.841334,0.258583,0.546111,0.194625,-0.350749,1.645962


In [10]:
# StandardScaler 선언 및 Fitting
StandardScaler = StandardScaler()
StandardScaler.fit(val_x)

# 데이터 변환
val_StandardScaled_data = StandardScaler.transform(val_x)

# 데이터 프레임으로 저장
val_StandardScaled_data = pd.DataFrame(val_StandardScaled_data)

In [11]:
val_normal, val_fraud = val_df['Class'].value_counts() # 정상치, 이상치
val_contamination = val_fraud / val_normal

In [12]:
val_contamination

0.0010551491277433877

In [13]:
test_x = test_df.drop(columns=['ID'])

In [16]:
# StandardScaler 선언 및 Fitting
StandardScaler = StandardScaler()
StandardScaler.fit(test_x)

# 데이터 변환
test_StandardScaled_data = StandardScaler.transform(test_x)

# 데이터 프레임으로 저장
test_StandardScaled_data = pd.DataFrame(test_StandardScaled_data)

# 단일 모델

## Elliptic Envelope

- 이전에 스케일링이 안 된 데이터를 적용했을 때 성능이 낮게 나왔지만, 스케일링이 된 데이터로 모델 학습 결과 성능이 향상됨  

In [ ]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(sdscaled_data)

val_pred_E = model.predict(val_StandardScaled_data) # model prediction
val_pred_E = get_pred_label(val_pred_E)
val_score_E = f1_score(val_y, val_pred_E, average='macro')

- `support_fraction`을 0.994로 설정한 이유는 파라미터 조정하며 성능 측정 결과 0.994가 좋았기 때문 ([참고](https://dacon.io/codeshare/5694))

In [ ]:
print(f'Validation F1 Score : [{val_score_E}]')
print(classification_report(val_y, val_pred_E))

Validation F1 Score : [0.8927516353661109]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.85      0.73      0.79        30

    accuracy                           1.00     28462
   macro avg       0.92      0.87      0.89     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
#best model : model "EllipticEnvelope"
test_pred_E = model.predict(test_StandardScaled_data) # model prediction
test_pred_E = get_pred_label(test_pred_E)

In [ ]:
test_pred_E

array([0, 0, 0, ..., 0, 0, 0])

## Isolation Forest

In [40]:
model = IsolationForest(contamination=val_contamination, random_state=42)
model.fit(sdscaled_data)

val_pred_IF = model.predict(val_StandardScaled_data) # model prediction
val_pred_IF = get_pred_label(val_pred_IF)
val_score_IF = f1_score(val_y, val_pred_IF, average='macro')

In [ ]:
print(f'Validation F1 Score : [{val_score_IF}]')
print(classification_report(val_y, val_pred_IF))

Validation F1 Score : [0.6720796742978351]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.36      0.33      0.34        30

    accuracy                           1.00     28462
   macro avg       0.68      0.67      0.67     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
test_pred_IF = model.predict(test_StandardScaled_data) # model prediction
test_pred_IF = get_pred_label(test_pred_IF)

In [ ]:
test_pred_IF

array([0, 0, 0, ..., 0, 0, 0])

## Local Outlier Factor

In [ ]:
model = LocalOutlierFactor(contamination=val_contamination)
model.fit(sdscaled_data)

LocalOutlierFactor(contamination=0.0010551491277433877)

In [ ]:
val_pred_LOF = model.fit_predict(val_StandardScaled_data) # model prediction
val_pred_LOF = get_pred_label(val_pred_LOF)
val_score_LOF = f1_score(val_y, val_pred_LOF, average='macro')

In [ ]:
print(f'Validation F1 Score : [{val_score_LOF}]')
print(classification_report(val_y, val_pred_LOF))

Validation F1 Score : [0.5322856805961294]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.06      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
test_pred_LOF = model.fit_predict(test_StandardScaled_data) # model prediction
test_pred_LOF = get_pred_label(test_pred_LOF)

In [ ]:
test_pred_LOF

array([0, 0, 0, ..., 0, 0, 0])

## Auto Encoder

In [41]:
## gpu 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [42]:
EPOCHS = 400
LR = 1e-2
BS = 16384 # 2^(x) 형태
SEED = 41

In [43]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(SEED) # seed 고정

In [ ]:
train_df.values

array([[ -1.35835406,  -1.34016307,   1.77320934, ...,  -0.05975184,
          4.98372109,  -0.99497175],
       [ -0.96627171,  -0.18522601,   1.79299334, ...,   0.06145763,
          1.41829106,  -0.99497175],
       [ -0.42596588,   0.96052304,   1.14110934, ...,   0.08108026,
         -0.25613079,  -0.99496   ],
       ...,
       [ -0.24192292,   0.71224653,   0.39980566, ...,   0.08126506,
         -0.23069936,   1.03490408],
       [  0.12031638,   0.93100513,  -0.54601214, ...,   0.06880333,
         -0.26982463,   1.03493932],
       [-11.88111789,  10.07178497,  -9.83478346, ...,   0.82373096,
         -0.29665339,   1.03495107]])

In [ ]:
train_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113837,-12.516732,10.187818,-8.476671,-2.510473,-4.586669,-1.394465,-3.632516,5.498583,4.893089,8.655320,...,-0.944759,-1.565026,0.890675,-1.253276,1.786717,0.320763,2.090712,1.232864,-0.169496,1.034857
113838,1.884849,-0.143540,-0.999943,1.506772,-0.035300,-0.613638,0.190241,-0.249058,0.666458,0.120908,...,0.144008,0.634646,-0.042114,-0.053206,0.316403,-0.461441,0.018265,-0.041068,0.530986,1.034881
113839,-0.241923,0.712247,0.399806,-0.463406,0.244531,-1.343668,0.929369,-0.206210,0.106234,-0.284708,...,-0.228876,-0.514376,0.279598,0.371441,-0.559238,0.113144,0.131507,0.081265,-0.230699,1.034904
113840,0.120316,0.931005,-0.546012,-0.745097,1.130314,-0.235973,0.812722,0.115093,-0.204064,-0.657422,...,-0.314205,-0.808520,0.050343,0.102800,-0.435870,0.124079,0.217940,0.068803,-0.269825,1.034939


In [44]:
class MyDataset(Dataset):
  def __init__(self,df,eval_mode):
    self.df = df
    self.eval_mode = eval_mode

    if self.eval_mode: ## val data의 경우에만 'Class' drop
      self.labels = self.df['Class'].values # label
      self.df = self.df.drop(columns=['Class']).values
    else:
      self.df = self.df.values

  def __getitem__(self,index):
    if self.eval_mode: ## val data
       x = torch.from_numpy(self.df[index]).type(torch.FloatTensor)
       y = torch.FloatTensor([self.labels[index]])
       return x, y

    else:
      self.x = self.df[index]
      return torch.Tensor(self.x)

  def __len__(self):
    return len(self.df)

In [45]:
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=6) # 1회당 BS개의 데이터씩 분할, 순서는 섞어서 분할(shuffle)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
valid_dataset = MyDataset(df=valid_df, eval_mode=True)
valid_loader = DataLoader(valid_dataset, batch_size=BS, shuffle=True, num_workers=6) # 1회당 BS개의 데이터씩 분할, 순서는 섞어서 분할(shuffle)

In [46]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.Encoder = nn.Sequential(
        nn.Linear(30,64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
    )
    self.Decoder = nn.Sequential( # encoder 복원
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        nn.Linear(64,30),
    )

  def forward(self,x):
    x = self.Encoder(x)
    x = self.Decoder(x)
    return x

In [47]:
class Trainer():
  def __init__(self,model,optimizer,train_loader,val_loader,scheduler,device):
    self.model = model
    self.optimizer = optimizer
    self.train_loader = train_loader
    self.val_loader = val_loader
    self.scheduler = scheduler
    self.device = device
    # loss function
    self.criterion = nn.L1Loss().to(self.device)

  def fit(self,):
    self.model.to(self.device)
    best_score = 0
    avg = 1 ##

    for epoch in range(EPOCHS):
      self.model.train()
      train_loss = []
      for x in iter(self.train_loader):
        x = x.float().to(self.device)
        self.optimizer.zero_grad()

        _x = self.model(x)
        loss = self.criterion(x, _x)

        loss.backward()
        self.optimizer.step()

        train_loss.append(loss.item())

      score = self.validation(self.model, 0.95)
      print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}]')

      if self.scheduler is not None:
        self.scheduler.step(score)

      if best_score <= score and avg > np.mean(train_loss):
        best_score = score
        avg = np.mean(train_loss)
        torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
        print('---------------------------')
        print(f'Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

  def validation(self, eval_model, thr):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    eval_model.eval()
    pred = []
    true = []

    with torch.no_grad():
      for x, y in iter(self.val_loader):
        x = x.float().to(self.device)

        _x = self.model(x)
        diff = cos(x,_x).cpu().tolist()
        batch_pred = np.where(np.array(diff)<thr,1,0).tolist()
        pred += batch_pred
        true += y.tolist()

    return f1_score(true, pred, average='macro')

In [ ]:
model = nn.DataParallel(AutoEncoder())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

In [ ]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.028220371742333685] Val Score : [0.8674887641844412]
---------------------------
Train loss : [0.028220371742333685] Val Score : [0.8674887641844412])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [1] Train loss : [0.028721494067992483] Val Score : [0.8674887641844412]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [2] Train loss : [0.029769121004002436] Val Score : [0.8674887641844412]


...

Epoch : [397] Train loss : [0.029551071247884204] Val Score : [0.8674887641844412]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [398] Train loss : [0.028448728578431264] Val Score : [0.8674887641844412]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [399] Train loss : [0.028400372713804245] Val Score : [0.8674887641844412]


In [51]:
model = AutoEncoder()
model.load_state_dict(torch.load('/content/drive/MyDrive/daicon/best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder(
    (Encoder): Sequential(
      (0): Linear(in_features=30, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
    (Decoder): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=30, bias=True)
    )
  )
)

In [48]:
def prediction(model, thr, test_loader, device):
  model.to(device)
  model.eval()

  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  pred = []

  with torch.no_grad():
    for x in iter(test_loader):
      x = x.float().to(device)
      _x = model(x)
      diff = cos(x, _x).cpu().tolist()
      batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
      pred += batch_pred

  return pred

In [49]:
valid_df = pd.read_csv('/content/drive/MyDrive/daicon/val.csv').drop(columns=['ID'])
valid_dff = valid_df.drop(columns=['Class'])

In [50]:
val_dataset = MyDataset(df=valid_dff, eval_mode=False)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=6)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [52]:
val_pred_AE = prediction(model, 0.95, val_loader, device)

In [53]:
val_pred_AE = np.array(val_pred_AE).reshape(-1)

In [54]:
val_pred_AE

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
val_score_AE = f1_score(val_y, val_pred_AE, average='macro')
print(f'Validation F1 Score : [{val_score_AE}]')
print(classification_report(val_y, val_pred_AE))

Validation F1 Score : [0.8674887641844412]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.66      0.83      0.74        30

    accuracy                           1.00     28462
   macro avg       0.83      0.92      0.87     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
test_df= test_df.drop(columns=['ID'])

In [ ]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
test_pred_AE = prediction(model, 0.95, test_loader, device)

# Ensemble

- Hard Voting(=Majority Voting) : 모델들이 결과를 예측하면 단순히 가장 많은 표를 얻은 결과를 선택  
- Soft Voting(=Probability Voting) : 각 클래스별로 모델들이 예측한 확률을 합산해 가장 높은 클래스 선택  

출처 : https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting  

<br>

---  
- 위 데이터는 비지도학습 데이터이지만, validation data에 label이 있기 때문에 간단한 voting 방법을 적용.  
- 여러 모델을 validation data에 학습한 결과를 모아 voting으로 해결. 예를 들어, 3가지 모델의 결과를 hard voting으로 앙상블할 경우, 두 가지 모델 이상이 클래스를 1이라 하면 해당 데이터의 클래스를 1로 봄  

(1) AE + EE + ISF

In [ ]:
# Hard voting ensemble
ensemble_pred = (val_pred_E + val_pred_IF + val_pred_AE) >= 2
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (hard voting): {ensemble_score}')

Ensemble F1 Score (hard voting): 0.8927516353661109


In [ ]:
# Soft voting ensemble
ensemble_pred = np.round((val_pred_E + val_pred_IF + val_pred_AE) / 3)
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (soft voting): {ensemble_score}')

Ensemble F1 Score (soft voting): 0.8927516353661109


In [ ]:
test_ensemble_pred = (test_pred_E + test_pred_IF + test_pred_AE) >= 2
ensemble_pred_int = test_ensemble_pred.astype(int)
submit = pd.read_csv('/content/drive/MyDrive/daicon/sample_submission.csv')
submit['Class'] = ensemble_pred_int
submit.to_csv('/content/drive/MyDrive/daicon/Ensemble_hard_1.csv', index=False)

(2) AE + EE + LOF

In [ ]:
# Hard voting ensemble
ensemble_pred = (val_pred_E + val_pred_LOF + val_pred_AE) >= 2
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (hard voting): {ensemble_score}')

Ensemble F1 Score (hard voting): 0.8791872484255181


(3) AE + EE + LOF + ISF

In [ ]:
# Hard voting ensemble
ensemble_pred = (val_pred_E + val_pred_IF + val_pred_AE + val_pred_LOF) >= 3
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (hard voting): {ensemble_score}')

Ensemble F1 Score (hard voting): 0.785556063429556


(4) AE + LOF + ISF

In [ ]:
# Hard voting ensemble
ensemble_pred = (val_pred_IF + val_pred_AE + val_pred_LOF) >= 2
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (hard voting): {ensemble_score}')

Ensemble F1 Score (hard voting): 0.7350567466625846


**voting 결과**  
- Auto Encoder, Elliptic Envelope, Isolation Forest를 앙상블한 결과 f1 score가 0.8928로 가장 높게 나왔다.  
- 단일 모델 중 가장 좋은 성능을 보인 Auto encoder보다 조금 향상됨(0.87 -> 0.8928).

# 모델 개선

**개선할 점**  
> 다른 모델
- Auto Encoder 기반 모델이 AE 뿐만 아니라 VAE, DAGMM(Deep Autoencoder Gaussian Mixture Model)도 존재.   
- One Class SVM

<br>

> 변수 선택  
- validation dataset과 class의 상관관계를 확인하여 모든 변수가 아닌 일부 변수만 선택하여 진행하기도 함  

<br>

> unsupervised learning  → supervised learning  
- (1) label 생성  
- (2) oversampling + classification  
※ private 3위 : MCD로 Label을 생성하고, Classification 모델인 lr, catboost 앙상블  
※ Private 2위 : MissForest, Oversampling, KNN
※ (2) 방식이 test 성능이 좋을 수 있어도 일반화 측면에서 신뢰하기 어려울 수 있음  


※ Oversampling  
- 낮은 비율 클래스의 데이터 수를 늘림으로써 데이터 불균형을 해소하는 방법  
- 대표적인 방법은 SMOTE(Synthetic Minority Over-samping Technique)이 있다.  

>SMOTE(Synthethic Minority Over-sampling Techinique)    

- SMOTE는 낮은 비율 클래스 데이터들의 최근접 이웃을 이용해 새로운 데이터를 생성하는 방법이다.  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FGaITt%2FbtqGmTl4AeX%2FqkrQ3yl4LjsNyDorwyr5Gk%2Fimg.png" height="200" width="500">

## One Class SVM

In [20]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

In [23]:
std_scaler = StandardScaler()
train_std = std_scaler.fit_transform(train_df)

pca = PCA(n_components=3)
prcomponent = pca.fit_transform(train_std)

In [24]:
principal = pd.DataFrame(data=prcomponent, columns = ['pca_v1', 'pca_v2', 'pca_v3'])

In [25]:
ocsvm = OneClassSVM(gamma=0.0001, kernel='rbf', max_iter=100000, nu=0.001, verbose=True)
ocsvm.fit(principal)

[LibSVM]

OneClassSVM(gamma=0.0001, max_iter=100000, nu=0.001, verbose=True)

In [27]:
valid_std = std_scaler.fit_transform(val_df)
prcomponent_ = pca.fit_transform(valid_std)
principal_ = pd.DataFrame(data=prcomponent_, columns = ['pca_v1', 'pca_v2', 'pca_v3'])

In [28]:
lab = val_df.iloc[:,-1]

In [31]:
def get_pred_label(model, x, k):
  prob = model.score_samples(x)
  prob = torch.tensor(prob, dtype = torch.float)
  topk_indices = torch.topk(prob, k = k, largest = False).indices

  pred = torch.zeros(len(x), dtype = torch.long)
  pred[topk_indices] = 1
  return pred.tolist(), prob.tolist()

In [32]:
val_pred_O, val_prob_O = get_pred_label(ocsvm, principal_, 28)
val_score_O = f1_score(lab, val_pred_O, average='macro')
print(f'Validation F1 Score : [{val_score_O}]')
print(classification_report(lab, val_pred_O))
tn, fp, fn, tp = confusion_matrix(lab, val_pred_O).ravel()
print('tp : ', tp, ', fp : ', fp, ', tn : ', tn, ', fn : ', fn)

Validation F1 Score : [0.9309641419574388]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.89      0.83      0.86        30

    accuracy                           1.00     28462
   macro avg       0.95      0.92      0.93     28462
weighted avg       1.00      1.00      1.00     28462

tp :  25 , fp :  3 , tn :  28429 , fn :  5


In [33]:
val_pred_O = np.array(val_pred_O)

In [34]:
def get_pred_label_(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [36]:
col_t = test_x.columns
test_df_std = std_scaler.fit_transform(test_x)
test_df_std_ = pd.DataFrame(data=test_df_std, columns=col_t)

prcomponent_ = pca.fit_transform(test_df_std_)
principal_ = pd.DataFrame(data=prcomponent_, columns = ['pca_v1', 'pca_v2', 'pca_v3'])

test_pred_O = ocsvm.predict(principal_)
test_pred_O = get_pred_label_(test_pred_O)

In [37]:
test_pred_O

array([0, 0, 0, ..., 0, 0, 0])

## OCSVM 이용해서 앙상블(AE + EE + ISF)

In [55]:
# Hard voting ensemble
ensemble_pred = (val_pred_O + val_pred_IF + val_pred_AE) >= 2
ensemble_score = f1_score(val_y, ensemble_pred, average='macro')
print(f'Ensemble F1 Score (hard voting): {ensemble_score}')

Ensemble F1 Score (hard voting): 0.9309641419574388
